In [2]:
import pandas as pd
from PIL import Image
import os
import numpy as np

df = pd.read_csv('full_df.csv')  
cols = ["ID", "Patient Age", "Patient Sex", "Left-Fundus", "Right-Fundus", "Left-Diagnostic Keywords", "Right-Diagnostic Keywords", 
        "filepath", "N", "D", "G", "C", "A", "H", "M", "O", "labels"]
df = df.drop(cols, axis=1)

df.head()

,target,filename
0,"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


In [3]:

import re
def sort_order(filename):
    match = re.match(r"(\d+)_(left|right)", filename)
    if match:
        number = int(match.group(1))
        next = match.group(2)
        nextPriority = 0 if next == 'right' else 1  
        return (number, nextPriority)
    else:
        return (float('inf'), 0)

df['sort_key'] = df['filename'].apply(sort_order)
df = df.sort_values('sort_key').drop(columns='sort_key')

df = df.reset_index(drop=True)

df.head(10)

,target,filename
0,"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,"[0, 0, 0, 1, 0, 0, 0, 0]",0_left.jpg
2,"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
3,"[1, 0, 0, 0, 0, 0, 0, 0]",1_left.jpg
4,"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
5,"[1, 0, 0, 0, 0, 0, 0, 0]",3_left.jpg
6,"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
7,"[0, 0, 0, 0, 0, 0, 0, 1]",4_left.jpg
8,"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg
9,"[0, 1, 0, 0, 0, 0, 0, 0]",5_left.jpg


In [4]:
from PIL import Image

def load_image(filename, img_dir):
    img_path = f"{img_dir}/{filename}"
    
    img = Image.open(img_path)
    
   
    img = img.resize((224, 224))
    
    img = np.array(img) / 255.0  
    
    return img

X = []
y = []

for idx, row in df.iterrows():
    filename = row['filename']
    target = row['target']
    
    img = load_image(filename, 'preprocessed_images') 

    X.append(img)
    y.append(target)

X = np.array(X)
y = np.vstack(y)  

print(X.shape)  
print(y.shape)  

img = Image.open('preprocessed_images/0_left.jpg')  
print(img.size)  



(6392, 224, 224, 3)
(6392, 1)
(512, 512)


In [5]:

import pandas as pd
import numpy as np
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder



label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.ravel())  
y = to_categorical(y_encoded, num_classes=8)


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(8, activation='relu'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

loss, accuracyy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {accuracyy * 100:.2f}%")

/Users/kirtitejgandham/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 173056)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    11,075,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,132,488 (42.47 MB)

 Trainable params: 11,132,488 (42.47 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 167s 1s/step - accuracy: 0.4383 - loss: 2.5110 - val_accuracy: 0.4464 - val_loss: 2.0428
Epoch 2/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 159s 989ms/step - accuracy: 0.4490 - loss: 2.2310 - val_accuracy: 0.4464 - val_loss: 2.0548
Epoch 3/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 155s 969ms/step - accuracy: 0.4485 - loss: 2.2494 - val_accuracy: 0.4464 - val_loss: 1.7251
Epoch 4/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 161s 1s/step - accuracy: 0.4444 - loss: 1.8483 - val_accuracy: 0.4464 - val_loss: 1.6928
Epoch 5/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 162s 1s/step - accuracy: 0.4543 - loss: 1.8225 - val_accuracy: 0.4464 - val_loss: 1.7764
Epoch 6/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 162s 1s/step - accuracy: 0.4517 - loss: 1.8492 - val_accuracy: 0.4464 - val_loss: 1.7028
Epoch 7/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.4415 - loss: 1.8018 - val_accuracy: 0.4464 - val_loss: 1.7060
Epoch 8/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 159s 985ms/step - accuracy: 0.4497 - loss: 1.8649 -

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Input, optimizers, callbacks
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from sklearn.utils.class_weight import compute_class_weight

DATA_DIR        = 'Organized_Images'
IMG_HEIGHT      = 224
IMG_WIDTH       = 224
BATCH_SIZE      = 8
VALID_SPLIT     = 0.2
SEED            = 42

EPOCHS_P1       = 2
LR_P1           = 5e-4      
EPOCHS_P2       = 3
LR_P2           = 5e-6     
UNFREEZE_LAYERS = 40

AUTOTUNE = tf.data.AUTOTUNE

raw_train = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR, labels='inferred', label_mode='categorical',
    validation_split=VALID_SPLIT, subset='training',
    seed=SEED, image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)
raw_val = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR, labels='inferred', label_mode='categorical',
    validation_split=VALID_SPLIT, subset='validation',
    seed=SEED, image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

class_names = raw_train.class_names
num_classes = len(class_names)
print("Classes:", class_names)

y = np.concatenate([y for _, y in raw_train], axis=0)
y_idx = np.argmax(y, axis=1)
cw = compute_class_weight('balanced', classes=np.arange(num_classes), y=y_idx)
class_weight_dict = dict(enumerate(cw))


train_ds_pp = raw_train.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
val_ds_pp   = raw_val  .map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)

aug_p1 = tf.keras.Sequential([layers.RandomFlip("horizontal")])

aug_p2 = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.10),
    layers.RandomZoom(0.05),
])

train_ds_p1 = train_ds_pp.map(lambda x, y: (aug_p1(x, training=True), y),
                              num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
train_ds_p2 = train_ds_pp.map(lambda x, y: (aug_p2(x, training=True), y),
                              num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds = val_ds_pp.prefetch(AUTOTUNE)


base_model = EfficientNetV2L(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(256, activation=None)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = Model(inputs, outputs)


best_model_file = "efficientnetv2l_fundus_best.h5"
cb = [
    callbacks.ModelCheckpoint(best_model_file,
                              monitor="val_accuracy",
                              save_best_only=True,
                              verbose=1),
    callbacks.ReduceLROnPlateau(monitor="val_accuracy",
                                patience=5,
                                factor=0.1,
                                min_lr=1e-7,
                                verbose=1),
    callbacks.EarlyStopping(monitor="val_accuracy",
                            patience=10,
                            verbose=1,
                            restore_best_weights=True)
]

base_model.trainable = False
print("Trainable params (head only):")
model.summary()

model.compile(
    optimizer=optimizers.Adam(LR_P1),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n PHASE 1: training head only (no class-weights)")
history_p1 = model.fit(
    train_ds_p1,
    validation_data=val_ds,
    epochs=EPOCHS_P1,
    callbacks=cb
)

for layer in base_model.layers[:-UNFREEZE_LAYERS]:
    layer.trainable = False
for layer in base_model.layers[-UNFREEZE_LAYERS:]:
    layer.trainable = True
    
print("\nTrainable layers in base_model after unfreeze:")
print(sum(l.trainable for l in base_model.layers), "of", len(base_model.layers))

model.compile(
    optimizer=optimizers.Adam(learning_rate=LR_P2),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print("\nPHASE 2(WARM-UP): fine-tuning (no class-weights)")
history_warm = model.fit(
    train_ds_p2,
    validation_data=val_ds,
    epochs=3,
    callbacks=cb
)

model.compile(
    optimizer=optimizers.Adam(learning_rate=LR_P2),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print("\nPHASE 2: fine-tuning with class-weights")
history_p2 = model.fit(
    train_ds_p2,
    validation_data=val_ds,
    epochs=EPOCHS_P2-3,
    class_weight=class_weight_dict,
    callbacks=cb
)

val_accs = (
    history_p1.history.get('val_accuracy', []) +
    history_warm.history.get('val_accuracy', []) +
    history_p2.history.get('val_accuracy', [])
)
best_val_acc = max(val_accs) if val_accs else 0
print(f"\nHighest Validation Accuracy Achieved: {best_val_acc:.4f}")

Python(1517) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Found 6392 files belonging to 8 classes.
Using 5114 files for training.
Found 6392 files belonging to 8 classes.
Using 1278 files for validation.
Classes: ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8']


2025-05-07 22:40:36.058768: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Trainable params (head only):


Model: "functional_10"

...

 Non-trainable params: 117,747,360 (449.17 MB)


 PHASE 1: training head only (no class-weights)
Epoch 1/2
640/640 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3868 - loss: 1.7300
Epoch 1: val_accuracy improved from -inf to 0.50704, saving model to efficientnetv2l_fundus_best.h5


640/640 ━━━━━━━━━━━━━━━━━━━━ 1174s 2s/step - accuracy: 0.3869 - loss: 1.7297 - val_accuracy: 0.5070 - val_loss: 1.3258 - learning_rate: 5.0000e-04
Epoch 2/2
640/640 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4973 - loss: 1.3293
Epoch 2: val_accuracy improved from 0.50704 to 0.55869, saving model to efficientnetv2l_fundus_best.h5


640/640 ━━━━━━━━━━━━━━━━━━━━ 1143s 2s/step - accuracy: 0.4973 - loss: 1.3293 - val_accuracy: 0.5587 - val_loss: 1.2016 - learning_rate: 5.0000e-04
Restoring model weights from the end of the best epoch: 2.

Trainable layers in base_model after unfreeze:
40 of 1028

PHASE 2(WARM-UP): fine-tuning (no class-weights)
Epoch 1/3
640/640 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4871 - loss: 1.4047
Epoch 1: val_accuracy did not improve from 0.55869
640/640 ━━━━━━━━━━━━━━━━━━━━ 1381s 2s/step - accuracy: 0.4871 - loss: 1.4046 - val_accuracy: 0.5509 - val_loss: 1.2185 - learning_rate: 5.0000e-06
Epoch 2/3
640/640 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5106 - loss: 1.3280
Epoch 2: val_accuracy improved from 0.55869 to 0.56416, saving model to efficientnetv2l_fundus_best.h5


640/640 ━━━━━━━━━━━━━━━━━━━━ 1356s 2s/step - accuracy: 0.5106 - loss: 1.3280 - val_accuracy: 0.5642 - val_loss: 1.2213 - learning_rate: 5.0000e-06
Epoch 3/3
640/640 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5164 - loss: 1.2831
Epoch 3: val_accuracy did not improve from 0.56416
640/640 ━━━━━━━━━━━━━━━━━━━━ 1355s 2s/step - accuracy: 0.5164 - loss: 1.2831 - val_accuracy: 0.5610 - val_loss: 1.2131 - learning_rate: 5.0000e-06
Restoring model weights from the end of the best epoch: 2.

PHASE 2: fine-tuning with class-weights

Highest Validation Accuracy Achieved: 0.5642


In [7]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, DenseNet201
from tensorflow.keras.layers import Concatenate
import tensorflow as tf
from tensorflow.keras import layers, Input, Model, optimizers, callbacks
from tensorflow.keras.applications import DenseNet201, ResNet50V2
from tensorflow.keras.applications.densenet import preprocess_input
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

DATA_DIR        = 'Organized_Images'
IMG_HEIGHT      = 224
IMG_WIDTH       = 224
BATCH_SIZE      = 8
VALID_SPLIT     = 0.2
SEED            = 42

EPOCHS_P1       = 2
LR_P1           = 5e-4      
EPOCHS_P2       = 4
LR_P2           = 5e-6     
UNFREEZE_LAYERS = 40

AUTOTUNE = tf.data.AUTOTUNE

raw_train = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR, labels='inferred', label_mode='categorical',
    validation_split=VALID_SPLIT, subset='training',
    seed=SEED, image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)
raw_val = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR, labels='inferred', label_mode='categorical',
    validation_split=VALID_SPLIT, subset='validation',
    seed=SEED, image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

class_names = raw_train.class_names
num_classes = len(class_names)
print("Classes:", class_names)

y = np.concatenate([y for _, y in raw_train], axis=0)
y_idx = np.argmax(y, axis=1)
cw = compute_class_weight('balanced', classes=np.arange(num_classes), y=y_idx)
class_weight_dict = dict(enumerate(cw))


train_ds_pp = raw_train.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
val_ds_pp   = raw_val  .map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)

aug_p1 = tf.keras.Sequential([layers.RandomFlip("horizontal")])

aug_p2 = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.10),
    layers.RandomZoom(0.05),
])

train_ds_p1 = train_ds_pp.map(lambda x, y: (aug_p1(x, training=True), y),
                              num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
train_ds_p2 = train_ds_pp.map(lambda x, y: (aug_p2(x, training=True), y),
                              num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds = val_ds_pp.prefetch(AUTOTUNE)

densenet_base = DenseNet201(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
resnet_base = ResNet50V2(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

x1 = densenet_base(inputs, training=False)
x1 = layers.GlobalAveragePooling2D()(x1)
x1 = layers.Dense(128)(x1) 
x2 = resnet_base(inputs, training=False)
x2 = layers.GlobalAveragePooling2D()(x2)
x2 = layers.Dense(128)(x2)  

x = layers.Concatenate()([x1, x2])

x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)


best_model_file = "densenet_resnet_fundus_best.h5"
cb = [
    callbacks.ModelCheckpoint(best_model_file,
                              monitor="val_accuracy",
                              save_best_only=True,
                              verbose=1),
    callbacks.ReduceLROnPlateau(monitor="val_accuracy",
                                patience=5,
                                factor=0.1,
                                min_lr=1e-7,
                                verbose=1),
    callbacks.EarlyStopping(monitor="val_accuracy",
                            patience=10,
                            verbose=1,
                            restore_best_weights=True)
]

densenet_base.trainable = False
resnet_base.trainable = False
print("Trainable params (head only):")
model.summary()

model.compile(
    optimizer=optimizers.Adam(LR_P1),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n PHASE 1: training head only (no class-weights)")
history_p1 = model.fit(
    train_ds_p1,
    validation_data=val_ds,
    epochs=EPOCHS_P1,
    callbacks=cb
)

for layer in densenet_base.layers[:-UNFREEZE_LAYERS]:
    layer.trainable = False
for layer in densenet_base.layers[-UNFREEZE_LAYERS:]:
    layer.trainable = True
    
resnet_unfreeze = min(UNFREEZE_LAYERS, len(resnet_base.layers) // 2)  # Adjust as needed
for layer in resnet_base.layers[:-resnet_unfreeze]:
    layer.trainable = False
for layer in resnet_base.layers[-resnet_unfreeze:]:
    layer.trainable = True
    
print("\nTrainable layers in base models after unfreeze:")
print(f"DenseNet: {sum(l.trainable for l in densenet_base.layers)} of {len(densenet_base.layers)}")
print(f"ResNet: {sum(l.trainable for l in resnet_base.layers)} of {len(resnet_base.layers)}")

model.compile(
    optimizer=optimizers.Adam(learning_rate=LR_P2),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print("\nPHASE 2(WARM-UP): fine-tuning (no class-weights)")
history_warm = model.fit(
    train_ds_p2,
    validation_data=val_ds,
    epochs=3,
    callbacks=cb
)

model.compile(
    optimizer=optimizers.Adam(learning_rate=LR_P2),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print("\nPHASE 2: fine-tuning with class-weights")
history_p2 = model.fit(
    train_ds_p2,
    validation_data=val_ds,
    epochs=EPOCHS_P2-3,
    class_weight=class_weight_dict,
    callbacks=cb
)

val_accs = (
    history_p1.history.get('val_accuracy', []) +
    history_warm.history.get('val_accuracy', []) +
    history_p2.history.get('val_accuracy', [])
)
best_val_acc = max(val_accs) if val_accs else 0
print(f"\nHighest Validation Accuracy Achieved: {best_val_acc:.4f}")


Found 6392 files belonging to 8 classes.
Using 5114 files for training.
Found 6392 files belonging to 8 classes.
Using 1278 files for validation.
Classes: ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8']


2025-05-07 15:07:59.121471: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Trainable params (head only):


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ densenet201         │ (None, 16, 16,    │ 18,321,984 │ input_layer_11[0… │
│ (Functional)        │ 1920)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50v2          │ (None, 16, 16,    │ 23,564,800 │ input_layer_11[0… │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1920)      │          0 │ densenet201[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50v2[0][0]  │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │    245,888 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │    262,272 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256)       │          0 │ dense_4[0][0],    │
│ (Concatenate)       │                   │            │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ concatenate_1[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 8)         │      2,056 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 42,398,024 (161.74 MB)

 Trainable params: 510,728 (1.95 MB)

 Non-trainable params: 41,887,296 (159.79 MB)


 PHASE 1: training head only (no class-weights)
Epoch 1/8
640/640 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.4462 - loss: 1.6180
Epoch 1: val_accuracy improved from -inf to 0.55869, saving model to densenet_resnet_fundus_best.h5


640/640 ━━━━━━━━━━━━━━━━━━━━ 5360s 8s/step - accuracy: 0.4463 - loss: 1.6177 - val_accuracy: 0.5587 - val_loss: 1.3306 - learning_rate: 5.0000e-04
Epoch 2/8
640/640 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.5654 - loss: 1.1967
Epoch 2: val_accuracy did not improve from 0.55869
640/640 ━━━━━━━━━━━━━━━━━━━━ 5125s 8s/step - accuracy: 0.5654 - loss: 1.1966 - val_accuracy: 0.4546 - val_loss: 1.2731 - learning_rate: 5.0000e-04
Epoch 3/8
 94/640 ━━━━━━━━━━━━━━━━━━━━ 56:03 6s/step - accuracy: 0.5643 - loss: 1.1336